# Analisando os dados da Olist

Para a nossa segunda parte da aula, iremos trabalhar em cima de uma base de dados disponibilizada pela Olist, uma empresa de e-commerce. Essa base de dados
é composta por 9 tabelas, conforme o seguinte esquema:

![](banco_de_dados_olist.png)

Essa base de dados é uma base menos "limpa" e é o que mais se aproxima da análise de dados do dia-a-dia de uma pessoa que trabalha com dados no mundo corporativo. Com essas tabelas, precisaremos demais manipulações de dados de forma mais minuciona que aquela que fizemos pela manhã. Vamos lá!

Para ter acesso à base de dados, acesse: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce


# Coleta dos Dados

Nossa base de dados da Olist é composta por 9 tabelas diferentes, nelas temos informações de:
- pedidos (olist_orders_dataset)
- itens dos pedidos (olist_order_items_dataset)
- review dos usuários sobre os pedidos (olist_order_reviews_dataset)
- detalhes de pagamento dos pedidos (olist_order_payments_dataset)
- detalhes do consumidor que fez os pedidos (olist_customers_dataset)
- detalhes de geolocalização do consumidor (olist_geolocation_dataset)
- detalhes dos produtos (olist_products_dataset)
- detalhes dos vendedores (olist_sellers_dataset)

Nesta aula, iremos focar em três datasets, o de pedidos (olist_orders_dataset), itens do pedido (olist_order_items_dataset) e o de consumidor (olist_customers_dataset). Vamos lá!

In [ ]:
import pandas as pd

In [ ]:
df_pedidos = pd.read_csv('olist_orders_dataset.csv')
df_pedidos

Para a tabela de pedidos, iremos utilizar as seguintes colunas:

- **order_id**: Identificador do pedido
- **customer_id**: Identificador do cliente. Essa coluna é importante pois será a nossa chave com a tabela de consumidor
- **order_status**: Status do pedido
- **order_purchase_timestamp**: timestamp do horário que a compra foi feita
- **order_delivered_customer_date**: timestamp do horário que a compra foi entregue ao cliente

In [ ]:
df_itens_pedido = pd.read_csv('dataset_olist/olist_order_items_dataset.csv')
df_itens_pedido

Para a tabela de itens dos pedidos, iremos utilizar as seguintes colunas:

- **order_id**: Identificador do pedido. Será a nossa chave com a tabela de pedidos
- **product_id**: Identificador do produto
- **seller_id**: Identificador do vendedor
- **shipping_limit_date**: timestamp limite para a entrega do pedido
- **price**: valor do pedido
- **freight_value**: valor do frete

In [ ]:
df_consumidor = pd.read_csv('dataset_olist/olist_customers_dataset.csv')
df_consumidor

Para a tabela de consumidor, iremos utilizar as seguintes colunas:

- **customer_id**: Identificador do cliente. Essa coluna é importante pois será a nossa chave com a tabela de pedidos. 
- **customer_state**: Estado
- **customer_city**: Cidade

# Objetivo da nossa Análise Exploratória:

Lembra que pela manhã comentamos a necessidade de estabelecer nossas perguntas para direcionar nossas análises? Para essa segunda parte da aula, buscaremos responder perguntas simples com base nessas 3 tabelas:

- Qual a frequência de compra dos clientes na plataforma da Olist no ano de 2017?
- Qual é o valor médio das compras desses clientes?  
- O número de vendas varia de acordo com a época do ano?
- O tempo limite de entrega das compras está sendo respeitado?

# Limpando os dados da Olist

## Pedidos

In [ ]:
# Vamos entender um pouquinho melhor dos nossos dados
df_pedidos.info()

Estamos lidando com um dataset relativamente grande, com quase 100.000 linhas. Temos dados nulos nas colunas de data,
o que faz sentido, pois nem todo o pedido realizado foi devidamente pago e finalizado, então aqueles pedidos que por algum motivo não foram finalizados. Por isso, algumas informações não estarão disponíveis. Por exemplo: um pedido que teve o pagamento negado, não terá dados de entrega. 

In [ ]:
# Vamos começar nossa limpeza filtrando somente as colunas que desejamos utilizar
columns_drop = ['order_approved_at', 'order_delivered_carrier_date', 'order_estimated_delivery_date']
df_pedidos = df_pedidos.drop(columns_drop, axis=1)
df_pedidos

In [ ]:
# Agora iremos filtrar somente aqueles pedidos que foram finalizados, mas antes é importante entender todos os status disponíveis que temos na base
df_pedidos['order_status'].unique()

In [ ]:
# filtrando apenas os pedidos que foram entregues
df_pedidos = df_pedidos.loc[df_pedidos['order_status'] == 'delivered']
df_pedidos

In [ ]:
# Filtrando somente pedidos finalizados, percebemos que perdemos um número razoável de linhas, cerca de 3 mil.
# Agora, com nosso dataset filtrado, precisamos resetar o seu index:
df_pedidos = df_pedidos.reset_index(drop=True)
df_pedidos

In [ ]:
# Para fecharmos a limpeza dessa tabela, precisamos criar uma coluna de mês! Uma das perguntas que precisamos responder é se existe sazonalidade
# nas vendas, então, criar uma coluna de mês irá facilitar bastante nosso trabalho na etapa de análise:

# Vamos transformar nossas colunas de data (que estão como string) em data: 
df_pedidos['order_purchase_timestamp'] = pd.to_datetime(df_pedidos['order_purchase_timestamp'])
df_pedidos['order_delivered_customer_date'] = pd.to_datetime(df_pedidos['order_delivered_customer_date'])

# Agora sim, vamos criar uma coluna de mês!
df_pedidos['reference_month'] = df_pedidos['order_purchase_timestamp'].dt.strftime('%Y-%m-01')
df_pedidos

## Itens do Pedido

In [ ]:
# Vamos entender um pouquinho melhor dos nossos dados
df_itens_pedido.info()

Estamos lidando com um dataset um pouco maior que o de pedidos, o que faz sentido, pois nossa informação aqui está a nível de item, e um pedido pode ter 1 ou mais itens. Não temos dados nulos nessa tabela, mas isso não significa que aqui constam somente pedidos confirmados, tá? Precisaremos filtrá-los em breve. 

## Consumidor

In [ ]:
# Vamos entender um pouquinho melhor dos nossos dados
df_consumidor.info()

In [ ]:
df_consumidor = df_consumidor.drop('customer_unique_id', axis=1)
df_consumidor

## Juntando os três datasets

Agora que fizemos a limpeza dos nossos três datasets, chegou a hora de juntarmos de acordo com a coluna em comum em cada um deles. Um ponto importante 
aqui é que queremos manter as informações que correspondem somente à pedidos **finalizados**. 

Para unir os 3 datasets usaremos a função merge(), precisamos entender qual é a coluna em comum entre os 3: no nosso caso, serão as colunas customer_id e order_id. 



In [ ]:
# Por padrão é realizado o merge dos dados com o inner join
# Jutando os df pedidos e itens de pedidos
df_olist = pd.merge(df_pedidos, df_itens_pedido, on='order_id')
df_olist

In [ ]:
# Jutando agora com os dados de consumidor
df_olist = pd.merge(df_olist, df_consumidor, on='customer_id')
df_olist

In [ ]:
# Exportando a base final (p/ encaminhar para as alunas)
df_olist.to_csv('base_final_s14_olist.csv', index=False)

# Analisando os dados

Agora chegou a hora de respondermos as perguntas que estipulamos no começo dessa análise:
- Qual é o valor médio das compras desses clientes?  
- O número de vendas varia de acordo com a época do ano?
- O tempo limite de entrega das compras está sendo respeitado?

### Qual é o valor médio das compras desses clientes? E qual o valor médio pago em frete?


In [ ]:
# Aqui estamos buscando uma estatística descritiva, podemos utilizar a nossa função describe() para trazer essas informações
df_olist.describe().T

Apesar de termos pedidos com valores bem altos, como o caso do pedido no valor de R$ 6735, a média dos pedidos está cerca de 120 reais. O frete segue o mesmo padrão, sendo cerca de 10% do valor do pedido. 

Fazendo uma análise adicional aqui, pela coluna order_item_id, conseguimos observar a quantidade de itens que temos dentro de um pedido. A média de cada um desses pedidos é ter cerca de 1,2 itens. Sendo o seu máximo, um pedido que possui 21 itens iguais. 

### O número de vendas varia de acordo com a época do ano?

![](calendario-sazonalidades.jpg)
Fonte: https://pagar.me/blog/como-lidar-com-a-sazonalidade-de-vendas-no-e-commerce/


As sazonalidades podem ser positivas ou negativas, e essa imagem pode nos ajudar a interpretar nossos dados.

In [ ]:
# Agora, vamos fazer uma análise de sazonalidade. Será que o número de vendas aumenta ou diminui de acordo com a época do ano?
df_sazonalidade = df_olist.groupby('reference_month').agg({'order_id': 'nunique'})
df_sazonalidade.sort_values(by='order_id', ascending=False)

### O tempo limite de entrega das compras está sendo respeitado?


In [ ]:
# Para essa análise, iremos criar uma classificação para o status do frete, sendo: ATRASADO, ADIANTADO e DENTRO DO ESPERADO, e então iremos observar a proporção
# dos pedidos para cada status.

df_olist['order_purchase_timestamp'] = pd.to_datetime(df_olist['order_purchase_timestamp'])
df_olist['order_delivered_customer_date'] = pd.to_datetime(df_olist['order_delivered_customer_date'])

df_olist['order_delivered_date'] = df_olist['order_delivered_customer_date'].dt.date
df_olist['shipping_limit_dt'] = pd.to_datetime(df_olist['shipping_limit_date']).dt.date

df_olist.loc[df_olist['order_delivered_date'] > df_olist['shipping_limit_dt'], 'status_entrega'] = 'ATRASADO'
df_olist.loc[df_olist['order_delivered_date'] < df_olist['shipping_limit_dt'], 'status_entrega'] = 'ADIANTADO'
df_olist.loc[df_olist['order_delivered_date'] == df_olist['shipping_limit_dt'], 'status_entrega'] = 'DENTRO DO ESPERADO'
df_olist

In [ ]:
df_olist["status_entrega"].value_counts(normalize=True)

# Ou seja, cerca de 72% dos pedidos chegam atrasados na casa dos consumidores. É interessante entender o padrão aqui, esse status se mantém de acordo com 
# a cidade? O quanto esse status mudaria se olhassemos para uma cidade grande tipo São Paulo? 

# Nota para a professora: Esses questionamentos colocamos após as análises não devem ser resolvidos por você. A ideia aqui é que eles sejam feitos para instigar o senso investigativo
# das alunas e ajudá-las a pensar em perguntas para a dinâmica a seguir.

# Dinâmica: Explorando os dados da Olist

Para essa dinâmica, vocês terão 30 minutos para continuar explorando a base que criamos durante essa aula. A ideia é que vocês estipulem 3 a 5 perguntas simples (diferentes da que fizemos em aula) e faça as análises necessárias para chegar em sua resposta. 

In [ ]:
# estado com maior quantidade de compra
df_olist['customer_state'].value_counts()

In [ ]:
# quais são os 5 estados que mais compram?
df_olist['customer_state'].value_counts(normalize=False).head(5)

In [ ]:
# 
df_olist['status_entrega'].value_counts()

In [ ]:
df_olist['reference_month'] = df_olist['order_purchase_timestamp'].dt.month
df_olist

In [ ]:
df_olist.groupby(['status_entrega', 'reference_month']).agg({'order_id': 'count'}).reset_index().sort_values('order_id', ascending=False)

# Dinâmica: Visualizações dos dados da Olist

Para essa dinâmica, vocês terão 40 minutos para criar as visualizações da base que criamos nessa aula (e com qualquer incremento que vocês fizeram à base na dinâmica anterior). 

Crie visualizações que te ajudem à responder as perguntas que vocês fizeram na dinâmica anterior. Utilize os conhecimentos da base da Netflix que fizemos pela manhã para criar seus próprios gráficos.


# Desafio da Semana

Objetivo: Sedimentar os conhecimentos de Análise de Dados que aprendemos nessa aula. 

Desafio: Criar um notebook de análise exploratória (como fizemos na nossa aula de hoje) com todas as etapas de coleta, limpeza, análise e visualização com base de dados da Olist.

Requisitos: 
- Formular o mínimo de 4 perguntas para responder com suas análises;
- Utilizar pelo menos 3 bases de dados da Olist (caso você deseje criar sua base do zero). Caso deseje continuar utilizando a que criamos em aula, é necessário incluir pelo menos mais 2 tabelas para enriquecer sua análise.   
- Criar pelo menos 3 gráficos.
- Exporte sua base final.


Submeta uma pasta que contenha: 
- o arquivo .ipynb com sua análise exploratória;
- a base final criada por você no formato .csv;
- as bases da olist em .csv que você utilizou na criação da sua analise exploratória.
